# **Thư viện**

In [1]:
from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
# from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

import time
from bs4 import BeautifulSoup
import json
from urllib.parse import urlparse
import csv

import pandas as pd

# **Lấy source page trang `https://mobilecity.vn/dien-thoai`**

In [2]:
# Get current time
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Print the current time
print("Thời gian bắt đầu thu thập:", current_time)

Thời gian bắt đầu thu thập: 2024-11-25 03:33:49


In [3]:
# Tạo instance của trình duyệt
driver = webdriver.Chrome()

# Phóng to trình duyệt
driver.maximize_window()

# **Thu thập đường dẫn đến từng trang điện thoại chi tiết**

In [4]:
data = pd.read_csv("links.csv")
links = data.links.to_list()

In [5]:
def is_valid_url(url):
    """Check if the URL is valid."""
    parsed = urlparse(url)
    return bool(parsed.scheme and parsed.netloc)

# **Thu thập các thuộc tính của từng sản phẩm**

In [6]:
product_data = []
counter = 0  
# wait = WebDriverWait(driver, 3)

In [7]:
for i in range(counter, len(links)):
    links[i] = links[i].strip()  # Xóa khoảng trắng đầu và cuối
    if not is_valid_url(links[i]):
        print(f"Invalid URL: {links[i]}")
        continue  # Skip invalid URLs
        
    driver.get(links[i])
    time.sleep(3)  # Chờ trang tải

    # Lấy HTML trang hiện tại
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Thu thập tên sản phẩm
    try:
        product_title = soup.find("h1", class_="title").text.strip()
    except AttributeError:
        product_title = "Không có tên sản phẩm"

    # Thu thập hãng điện thoại
    second_li = driver.find_element(By.XPATH, '//ul[@itemscope]/li[2]/a/span')
    brand = second_li.text.strip()

    # Lấy thông tin về thời gian bảo hành
    try:
        warranty_text = soup.find('span', class_='warranty-content-default').get_text(strip=True)
    except AttributeError:
        warranty_text = "Không có thông tin bảo hành"

    # Lấy thông số kỹ thuật
    try:
        specs_table = soup.find('div', class_='product-info-content').find('table')
        specs = {}
        if specs_table:
            rows = specs_table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    specs[cols[0].text.strip()] = cols[1].text.strip()
    except AttributeError:
        specs = {"Không có thông số kỹ thuật": "N/A"}

    # Thu thập thông tin đánh giá
    try:
        rating = soup.find('div', class_='comment-vote__star-number').get_text(strip=True)
    except AttributeError:
        rating = "Không có đánh giá"

    # Thu thập thông tin số lượt đánh giá và hỏi đáp
    try:
        total_reviews = soup.find('div', class_='comment-vote__star-total').get_text(strip=True)
        total_reviews = total_reviews.replace("đánh giá và hỏi đáp", "").strip()
    except AttributeError:
        total_reviews = "Không có thông tin đánh giá và hỏi đáp"

    ##### Màu sắc + Phiên bản bộ nhớ + Giá tương ứng

    color_storage_price = []
    color_items = driver.find_elements(By.CLASS_NAME, "color-item.attribute-item")
    storage_items = driver.find_elements(By.CLASS_NAME, "storage-item.attribute-item")

    if len(color_items) == 0 and len(storage_items) == 0:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        try:
            price_new = soup.find("p", class_="price").text.strip()
        except AttributeError:
            price_new = "Không có giá mới"
        try:
            price_old = soup.find("p", class_="price-old").text.strip()
        except AttributeError:
            price_old = "Không có giá cũ"
        color_storage_price.append(["", "", price_new, price_old])
    elif len(color_items) == 0:
        for storage in storage_items:
            storage.click()
            storage_name = storage.text.strip()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            try:
                price_new = soup.find("p", class_="price").text.strip()
            except AttributeError:
                price_new = "Không có giá mới"
            try:
                price_old = soup.find("p", class_="price-old").text.strip()
            except AttributeError:
                price_old = "Không có giá cũ"
            color_storage_price.append(["", storage_name, price_new, price_old])
    elif len(storage_items) == 0:
        for color in color_items:
            color.click()
            color_name = color.get_attribute("data-title")
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            try:
                price_new = soup.find("p", class_="price").text.strip()
            except AttributeError:
                price_new = "Không có giá mới"
            try:
                price_old = soup.find("p", class_="price-old").text.strip()
            except AttributeError:
                price_old = "Không có giá cũ"
            color_storage_price.append([color_name, "", price_new, price_old])
    else:
        for color in color_items:
            for storage in storage_items:
                color.click()
                storage.click()
                color_name = color.get_attribute("data-title")
                storage_name = storage.text.strip()
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                try:
                    price_new = soup.find("p", class_="price").text.strip()
                except AttributeError:
                    price_new = "Không có giá mới"
                try:
                    price_old = soup.find("p", class_="price-old").text.strip()
                except AttributeError:
                    price_old = "Không có giá cũ"    
                color_storage_price.append([color_name, storage_name, price_new, price_old])

    # # Thu thập màu sắc
    # try:
    #     color_items = soup.find_all("div", class_="color-item")
    #     colors = [color['data-title'] for color in color_items]
    # except AttributeError:
    #     colors = ["Không có màu sắc"]

    # # Lấy thông tin về các phiên bản bộ nhớ
    # try:
    #     storage_items = soup.find_all('div', class_='storage-item attribute-item')
    #     storage_options = [item.text.strip() for item in storage_items]
    # except AttributeError:
    #     storage_options = ["Không có phiên bản bộ nhớ"]

    # # Thu thập giá sản phẩm (giá mới và giá cũ)
    # try:
    #     price_new = soup.find("p", class_="price").text.strip()
    # except AttributeError:
    #     price_new = "Không có giá mới"

    # try:
    #     price_old = soup.find("p", class_="price-old").text.strip()
    # except AttributeError:
    #     price_old = "Không có giá cũ"

    #####

    product_data.append({
        "Tên sản phẩm": product_title,
        "Loại điện thoại": brand,
        # "Giá mới": price_new,
        # "Giá cũ": price_old,
        # "Màu sắc": ", ".join(colors),
        # "Các phiên bản bộ nhớ": ", ".join(storage_options),
        "Màu sắc - Phiên bản bộ nhớ - Giá tương ứng": color_storage_price,
        "Thời gian bảo hành": warranty_text,
        "Thông số kỹ thuật": json.dumps(specs, ensure_ascii=False),
        "Đánh giá": rating,
        "Số lượt đánh giá và hỏi đáp": total_reviews,
        "Đường dẫn": links[i],
    })
    counter +=1
    # if counter % 100 == 0:
    #     print(f"Đã thu thập thành công {counter} sản phẩm")
    print(f"Đã thu thập: {counter}/{len(links)} sản phẩm", end="\r")

# In thông báo khi hoàn thành
print("Hoàn thành!!!")

Hoàn thành!!!1420/1420 sản phẩm


In [8]:
# Get current time
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Print the current time
print("Thời gian kết thúc thu thập:", current_time)

Thời gian kết thúc thu thập: 2024-11-25 05:36:21


# **Lưu dữ liệu thu thập được vào file `phone_data.csv`**

In [9]:
# Lưu dữ liệu vào file CSV
with open('data.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = product_data[0].keys()  # Lấy các tên trường từ phần tử đầu tiên
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in product_data:
        writer.writerow(data)

# Đóng trình duyệt
driver.quit()